In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# --- SOLUCIÓN AL ERROR ---
# Aumentamos el límite de embed a 50 MB
plt.rcParams['animation.embed_limit'] = 50.0 

# --- 1. Generación de la Señal Sintética ---
T_TRUE = 10.0      
T_DUR = 1.0        
DEPTH = 0.05       
TOTAL_TIME = 60.0  
N_POINTS = 1200
t_all = np.linspace(0, TOTAL_TIME, N_POINTS)
flux_all = np.ones_like(t_all) + np.random.normal(0, 0.002, N_POINTS)

for i in range(int(TOTAL_TIME / T_TRUE) + 1):
    mask = np.abs(t_all - i * T_TRUE) < T_DUR / 2
    flux_all[mask] -= DEPTH

# --- 2. Configuración de la Figura ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9, 8))
plt.subplots_adjust(hspace=0.3)


ax1.plot(t_all, flux_all, 'k.', markersize=1, alpha=0.3)
line_time = ax1.axvline(x=0, color='red', lw=1.5)
ax1.set_title("Serie Temporal")
ax1.set_xlim(0, TOTAL_TIME)

scat_accum = ax2.scatter([], [], c='blue', s=8, alpha=0.4, label='Datos faseados')
scat_curr = ax2.scatter([], [], c='red', s=40, edgecolors='k', zorder=10, label='Punto actual')
txt_title = ax2.set_title("", fontweight='bold')
ax2.set_xlim(0, 1)
ax2.set_ylim(0.93, 1.02)
ax2.set_xlabel("Fase Normalizada")
ax2.legend(loc='lower left')

# --- 3. Lógica de la Animación OPTIMIZADA ---
# Reducimos frames para que sea más ligero (150 por sección es suficiente para visualizar)
frames_per_test = 150 
periods = [T_TRUE, T_TRUE * 2, T_TRUE / 2]
labels = [f"Periodo Real ($T={T_TRUE}$)", f"Múltiplo ($2T={T_TRUE*2}$)", f"Submúltiplo ($T/2={T_TRUE/2}$)"]

x_data, y_data = [], []

def update(frame):
    global x_data, y_data
    
    section = frame // frames_per_test
    idx = frame % frames_per_test
    
    if idx == 0: 
        x_data, y_data = [], []
        
    P_test = periods[section]
    
    # Muestreo optimizado: recorremos el tiempo proporcionalmente a los frames
    # Usamos interpolación de índices para cubrir todo el TOTAL_TIME en menos frames
    data_idx = int((idx / frames_per_test) * len(t_all))
    if data_idx >= len(t_all): data_idx = len(t_all) - 1
    
    t_curr = t_all[data_idx]
    f_curr = flux_all[data_idx]
    
    # Fase centrada
    phase_curr = ((t_curr + P_test/2) % P_test) / P_test
    
    x_data.append(phase_curr)
    y_data.append(f_curr)
    
    line_time.set_xdata([t_curr])
    scat_accum.set_offsets(np.c_[x_data, y_data])
    scat_curr.set_offsets(np.c_[[phase_curr], [f_curr]])
    txt_title.set_text(f"Plegando con {labels[section]}")
    
    return line_time, scat_accum, scat_curr, txt_title

    plt.rcParams.update({
    "text.usetex": False,            
    "font.family": "serif",
    "font.serif": ["Latin Modern Roman", "Computer Modern Serif", "serif"],
    "mathtext.fontset": "cm",        
    "axes.labelweight": "normal"
})

plt.rcParams.update({
    "xtick.labelsize": 13,
    "ytick.labelsize": 13
})

writer = animation.FFMpegWriter(fps=30, metadata=dict(artist='Jorge Acebes'), bitrate=2000)

ani = animation.FuncAnimation(fig, update, frames=frames_per_test * 3, interval=50, blit=True)

# dpi=300 define la resolución final efectiva
ani.save("transito_fold.mp4", writer=writer, dpi=300)
plt.close()
